In [1]:
import pandas as pd

In [2]:
import os
import sys
import logging

import numpy as np
# import pandas as pd 

from gensim.models import Word2Vec

import config
from ingredient_parser import ingredient_parser

sys.path.append('..')

if sys.platform == 'linux':
    path = config.LINUX_PATH
else:
    path = config.OS_PATH
# os.chdir(path)

In [4]:
data = pd.read_csv('input/df_parsed_new.csv')
data['parsed_new'] = data.ingredients.apply(ingredient_parser)
data.head()

,Srno,recipe_name,ingredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,recipe_urls,ingredients_parsed,parsed_new
0,1,Masala Karela Recipe,['6 Karela (Bitter Gourd/ Pavakkai) - deseeded...,15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"['karela', 'onion', 'flour', 'amchur mango', '...","[karela, onion, flour, amchur mango, sunflower..."
1,2,Spicy Tomato Rice (Recipe),"['2-1 / 2 cups rice - cooked', ' 3 tomatoes', ...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",http://www.archanaskitchen.com/spicy-tomato-ri...,"['rice', 'tomato', 'bc belle bhat', 'per', 'ch...","[rice, tomato, bc belle bhat, per, chickpea le..."
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"['1-1/2 cups Rice Vermicelli Noodles (Thin)', ...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",http://www.archanaskitchen.com/ragi-vermicelli...,"['rice vermicelli noodle', 'onion', 'carrot', ...","[rice vermicelli noodle, onion, carrot, pea, u..."
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"['500 grams Chicken', '2 Onion - chopped', '1 ...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,http://www.archanaskitchen.com/gongura-chicken...,"['chicken', 'onion', 'tomato', 'slit', 'masala...","[chicken, onion, tomato, slit, masala, sesame,..."
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"['1 tablespoon chana dal', ' 1 tablespoon whit...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"['chana dal', 'urad dal', 'chop', 'onion chop'...","[chana dal, urad dal, chop, onion chop, tomato..."


In [5]:
# get corpus with the documents sorted in alphabetical order
def get_and_sort_corpus(data):
    corpus_sorted = []
    for doc in data.parsed_new.values:
        doc.sort()
        corpus_sorted.append(doc)
    return corpus_sorted

corpus = get_and_sort_corpus(data)
print(f"Length of corpus: {len(corpus)}")

Length of corpus: 6303


In [6]:
# calculate average length of each document 
lengths = [len(doc) for doc in corpus]
avg_len = float(sum(lengths)) / len(lengths)
avg_len

6.856100269712835

In [7]:
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 8 # number of CPUs
window = 6 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100)

In [9]:
#Summarize the loaded model
print(model_cbow)

#Summarize vocabulary
words = list(model_cbow.wv.index_to_key)
words.sort()
# print(words)

#Acess vector for one word
# print(model_cbow.wv['chicken stock'])

# most similar
# model_cbow.wv.most_similar(u'cauliflower just larger than potato')
# model_cbow.wv.similarity('cauliflower', 'cauliflower just larger than potato')

Word2Vec<vocab=4467, vector_size=100, alpha=0.025>


In [10]:
model_cbow.save('models/model_cbow.bin')

In [11]:
class MeanEmbeddingVectorizer(object):

	def __init__(self, word_model):
		self.word_model = word_model
		self.vector_size = word_model.wv.vector_size

	def fit(self):  # comply with scikit-learn transformer requirement
		return self

	def transform(self, docs):  # comply with scikit-learn transformer requirement
		doc_word_vector = self.word_average_list(docs)
		return doc_word_vector

	def word_average(self, sent):
		"""
		Compute average word vector for a single doc/sentence.
		:param sent: list of sentence tokens
		:return:
			mean: float of averaging word vectors
		"""
		mean = []
		for word in sent:
			if word in self.word_model.wv.index_to_key:
				mean.append(self.word_model.wv.get_vector(word))

		if not mean:  # empty words
			# If a text is empty, return a vector of zeros.
			logging.warning("cannot compute average owing to no vector for {}".format(sent))
			return np.zeros(self.vector_size)
		else:
			mean = np.array(mean).mean(axis=0)
			return mean


	def word_average_list(self, docs):
		"""
		Compute average word vector for multiple docs, where docs had been tokenized.
		:param docs: list of sentence in list of separated tokens
		:return:
			array of average word vector in shape (len(docs),)
		"""
		return np.vstack([self.word_average(sent) for sent in docs])

In [12]:
# encode document by averaging word embeddings

# load model 
loaded_model = Word2Vec.load('models/model_cbow.bin')
if loaded_model:
    print("Successfully loaded model")

mean_vec_tr = MeanEmbeddingVectorizer(loaded_model)
doc_vec = mean_vec_tr.transform(corpus)

Successfully loaded model
